
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
# Make sure that all your data is numpy arrays and not pandas dataframes or series.
data = pd.read_csv('data/Corona_NLP_train.csv', encoding='latin1') 
# Remove data that are ckassified as neutral
data = data[data['Sentiment'] != 'Neutral']
# Classes in the data ['Positive' 'Extremely Negative' 'Negative' 'Extremely Positive']
classes = data['Sentiment'].unique()
#print(classes)
# positive classes as 1 and rest as 0
data['Sentiment'] = data['Sentiment'].map(lambda x: 1 if x in ['Positive', 'Extremely Positive'] else 0)
# Check the data if they are correct
#data.head()
X = data[[ "OriginalTweet"]].values
# Transform a 2D array into a 1D array 
X = np.reshape(X, (1,np.product(X.shape)))[0]
#X.head()
# Transform a 2D array into a 1D array 
Y = data[[ "Sentiment"]].values
Y = np.reshape(Y, (1,np.product(Y.shape)))[0]

n = len(X)
train_size, test_size = int(0.6 * n), int(0.15 * n)

X_train, Y_train = X[:train_size], Y[:train_size]
X_test, Y_test = X[train_size:train_size + test_size], Y[train_size:train_size + test_size]
X_valid, Y_valid = X[train_size + test_size:], Y[train_size + test_size:]

X_train = X_train
Y_train = Y_train
X_test = X_test
Y_test = Y_test
X_valid = X_valid
Y_valid = Y_valid

#print(X_train.shape)
#print(Y_train.shape)
#print(X_test.shape)
#print(Y_test.shape)
#print(X_valid.shape)
#print(Y_valid.shape)

#print(X.shape)
#print(Y.shape)


In [2]:

# Part 2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

model = Pipeline([
    ('vectorizer', CountVectorizer()),  
    ('classifier', LogisticRegression(max_iter=800, class_weight=True, random_state=42))])
model.fit(X_train, Y_train)

# Predict proba works
#model.predict_proba(X_valid)



Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier',
                 LogisticRegression(class_weight=True, max_iter=800,
                                    random_state=42))])

In [3]:

# Part 3
from sklearn.metrics import precision_score, recall_score
# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

pred_test = model.predict(X_test)

precision_0 = precision_score(Y_test, pred_test, pos_label=0)
precision_1 = precision_score(Y_test, pred_test, pos_label=1)
recall_0 = recall_score(Y_test, pred_test, pos_label=0)
recall_1 = recall_score(Y_test, pred_test, pos_label=1)

#print(precision_0)
#print(precision_1)
#print(recall_0)
#print(recall_1)
#print(results)
# Proba returns only values between 0 and 1 
probab = model.predict_proba(X_valid)
#print(sum(probab > 0))  
#print(sum(probab < 1))

In [4]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    if model.predict_proba is None: 
        raise Exception("Predict proba doesn't exist")
    cost = 0 
    predictions_proba = model.predict_proba(X)[:, 1] 

    for proba, y in zip(predictions_proba, Y):
        
        predictions = 1 if proba >= threshold else 0
        
        if predictions==1 and y==0:
            cost+=5
        elif predictions==0 and y==1:
            cost+=1
        else: 
            cost+=0
    # Find the average 
    avg_cost = cost/len(predictions_proba)
    return avg_cost


In [5]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

def optimal_threshold(model, X, Y):
    # try different sets of thresholds
    thresholds_test = np.linspace(0, 1, 50)  
    cost_test = []

    for i in thresholds_test:
        # calculate the cost using the function
        cost_i = cost(model, i, X, Y)
        cost_test.append(cost_i)
    # keep the optimal value
    optimal_i = np.argmin(cost_test)
    optimal_threshold = thresholds_test[optimal_i]
    optimal_threshold_cost = cost_test[optimal_i]
    
    return optimal_threshold, optimal_threshold_cost


optimal_threshold, optimal_threshold_cost = optimal_threshold(model, X_test, Y_test)

optimal_threshold = optimal_threshold
cost_at_optimal_threshold = optimal_threshold_cost

#print(optimal_threshold)
#print(optimal_threshold_cost)


In [10]:

# Part 6
import math
from math import sqrt

val_cost = cost(model, optimal_threshold, X_valid, Y_valid)

n = len(X_valid) 
a=0.01 #conf level = 1-a = 0.99
a_h=0.005
#first_component = -1/(2*n)
#second_component = math.log(a_h)
# compute Hoeffding's bound
#e = sqrt(-log(a/2)/(2*n)) 
#e = sqrt((first_component)*(second_component))
e = sqrt(((5 - 0) ** 2 * math.log(2 / 0.01)) / (2 * n))

lower_bound = val_cost-e
upper_bound = val_cost+e

cost_at_optimal_threshold_valid = val_cost
cost_interval_valid = (lower_bound, upper_bound)

#print(cost_at_optimal_threshold_valid)
#print(cost_interval_valid)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)
#print(np.mean(val_cost))
#print(val_cost)
#print(lower_bound) #0.25785262262864495
#print(upper_bound) #0.29345089327664087


In [7]:

# Part 7

from sklearn.metrics import log_loss

proba_valid = model.predict_proba(X_valid)[:, 1] 
pred_valid = (proba_valid >= optimal_threshold).astype(int)
# needs to be 1d array
Y_valid = np.array(Y_valid)  
Y_valid = Y_valid.flatten()  
# use the given formula
c = (1 - pred_valid) * Y_valid + 5 * (1 - Y_valid) * (pred_valid)
mean_c = np.mean(c)
variance_c = np.var(c)

n = len(c) 
a = 0.01
# compute Bennett's bound
e = np.sqrt(2 * variance_c * np.log(2/a)/n)  

lower_bound = mean_c - e
upper_bound = mean_c + e

variance_of_C = variance_c
interval_of_C = (lower_bound, upper_bound)

#print(variance_of_C)
#print(interval_of_C)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)
